In [ ]:
from gurobipy import *
from typing import Dict
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import RL
import importlib
import tool_func
from update import *

num_vehicle = 20
num_order = 40
num_city = 5
TIME = 30
CAPACITY = 7
row = [10, 3, 1, 10]
Vehicles ={}
speed = 20
cancel_penalty = 300
battery_consume = 10
battery_add = 300
speed = 20
matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city,0.3, (20,100))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order ={}
for time in range(TIME):
    group = [[],[]]
    for vehicle in Vehicles.values():
        if vehicle.whether_city:
            group[0].append(vehicle.id)
            
        else:
            group[1].append(vehicle.id)
    
    Orders = order_generator(num_order, time, num_city-1,CAPACITY,G)
    for order in Orders.values():
        orders_unmatched[order.id] = order
        Total_order[order.id] = order

    orders_virtual = orders_unmatched
    if time == 0:
        city_node = city_node_generator(G,orders_virtual,Vehicles,orders_unmatched)
    # order_virtual = RL.allocate(orders_unmatched) #此处强化学习
    if len(group[0]) != 0:
        temp_Lower_Layer = Lower_Layer(G,city_node ,Vehicles, orders_unmatched ,name,group,time)
        temp_Lower_Layer.get_decision()
        temp_Lower_Layer.constrain_1()
        temp_Lower_Layer.constrain_2()
        temp_Lower_Layer.constrain_3()
        temp_Lower_Layer.constrain_4()
        temp_Lower_Layer.constrain_5()
        temp_Lower_Layer.model.setParam('OutputFlag', 0)
        total_penalty = cancel_penalty*order_canceled
        temp_Lower_Layer.set_objective(matrix,total_penalty)
        temp_Lower_Layer.model.optimize()
        if temp_Lower_Layer.model.status == GRB.OPTIMAL:
            # 打印变量的最优值
            print(f"{time}次，Optimal solution:")
            i = 0
            with open(f"output_{time}.txt", "w") as file:
                for v in temp_Lower_Layer.model.getVars():
                    if v.x == 1:
                        file.write(f"{v.varName} = {v.x}\n")  # 将结果写入文件
                # 打印目标函数值
            print("Objective value:", temp_Lower_Layer.model.objVal)
            
            
        else:
            print(f"{time}次，No optimal solution found.")
            self_update(Vehicles)
        _,var_order = temp_Lower_Layer.get_decision()
        update_var(temp_Lower_Layer, Vehicles,orders_unmatched)
        vehicle_in_city = update_vehicle(Vehicles,battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched,time,speed)
        print(f"{len(orders_unmatched)}个订单未被匹配")
        print(f"{vehicle_in_city}辆汽车可调度")
       
    else:
        self_update(Vehicles)
        print(f"{time}次，没有车")
        vehicle_in_city = update_vehicle(Vehicles,battery_consume, battery_add, speed, G)
        order_canceled = update_order(orders_unmatched,time,speed)
        print(f"{len(orders_unmatched)}个订单未被匹配")
        print(f"{vehicle_in_city}辆汽车可调度")

Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


0次，Optimal solution:
Objective value: 200395.0
5个订单未被匹配
20辆汽车可调度
1次，No optimal solution found.
45个订单未被匹配
19辆汽车可调度
2次，No optimal solution found.
85个订单未被匹配
19辆汽车可调度
3次，No optimal solution found.
125个订单未被匹配
19辆汽车可调度
4次，No optimal solution found.
165个订单未被匹配
19辆汽车可调度
5次，No optimal solution found.
205个订单未被匹配
19辆汽车可调度
6次，No optimal solution found.
245个订单未被匹配
19辆汽车可调度
7次，No optimal solution found.
285个订单未被匹配
19辆汽车可调度
8次，No optimal solution found.
325个订单未被匹配
19辆汽车可调度
9次，No optimal solution found.
365个订单未被匹配
19辆汽车可调度
10次，No optimal solution found.
405个订单未被匹配
19辆汽车可调度
11次，No optimal solution found.
445个订单未被匹配
19辆汽车可调度
12次，No optimal solution found.
485个订单未被匹配
19辆汽车可调度
13次，No optimal solution found.
525个订单未被匹配
19辆汽车可调度
14次，No optimal solution found.
565个订单未被匹配
19辆汽车可调度
15次，No optimal solution found.
605个订单未被匹配
19辆汽车可调度
16次，No optimal solution found.
645个订单未被匹配
19辆汽车可调度
17次，No optimal solution found.
685个订单未被匹配
19辆汽车可调度
18次，No optimal solution found.
725个订单未被匹配
19辆汽车可调度
19次，No optimal solution foun

In [2]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        #print(vehicle)
        print(vehicle.get_history_decisions())

[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [4]:
for order in Total_order.values():
    print(order)

Order(id=0, passengers=7, departure=3, destination=1,virtual_departure=3, matched_vehicle_id=16,distance=120,battery=1206.8691718110124, status=Matched)
Order(id=1, passengers=1, departure=1, destination=2,virtual_departure=1, matched_vehicle_id=10,distance=77,battery=772.1721035441483, status=Matched)
Order(id=2, passengers=1, departure=3, destination=2,virtual_departure=3, matched_vehicle_id=12,distance=74,battery=749.3244438886136, status=Matched)
Order(id=3, passengers=5, departure=1, destination=2,virtual_departure=1, matched_vehicle_id=12,distance=77,battery=771.289174068208, status=Matched)
Order(id=4, passengers=6, departure=3, destination=2,virtual_departure=3, matched_vehicle_id=9,distance=74,battery=747.3245325383169, status=Matched)
Order(id=5, passengers=4, departure=2, destination=3,virtual_departure=2, matched_vehicle_id=8,distance=74,battery=742.2911107182416, status=Matched)
Order(id=6, passengers=2, departure=1, destination=3,virtual_departure=1, matched_vehicle_id=10

In [7]:
for vehicle in Vehicles.values():
    if vehicle.get_orders():
        for order in vehicle.get_orders():
            if order.matched_vehicle_id == 8:
                print(vehicle.id)
                

In [8]:
for order in Total_order.values():
    if order.matched_vehicle_id == 8:
        print(order)

Order(id=13, passengers=2, departure=5, destination=2,virtual_departure=5, matched_vehicle_id=8,battery=287.9966258615341, status=Matched)
Order(id=15, passengers=5, departure=4, destination=2,virtual_departure=4, matched_vehicle_id=8,battery=849.64585838219, status=Matched)


In [3]:
for vehicle in Vehicles.values():
    #if vehicle.id == 0:
        print(vehicle)
        #print(vehicle.get_history_decisions())

Vehicle(id=0, time=30, into_city=0, intercity=3, passenger=6, decision=0, battery=2562.758187336516, matched_order=1)
Vehicle(id=1, time=30, into_city=0, intercity=3, passenger=0, decision=2, battery=15543.564803599722, matched_order=0)
Vehicle(id=2, time=30, into_city=0, intercity=4, passenger=5, decision=0, battery=18197.698868844167, matched_order=1)
Vehicle(id=3, time=30, into_city=0, intercity=3, passenger=7, decision=0, battery=9955.494391128977, matched_order=2)
Vehicle(id=4, time=30, into_city=0, intercity=2, passenger=7, decision=0, battery=12208.933669551121, matched_order=1)
Vehicle(id=5, time=30, into_city=0, intercity=4, passenger=5, decision=0, battery=6746.859657316655, matched_order=1)
Vehicle(id=6, time=30, into_city=0, intercity=1, passenger=0, decision=2, battery=15557.053632882611, matched_order=0)
Vehicle(id=7, time=30, into_city=0, intercity=1, passenger=0, decision=2, battery=3911.491778356056, matched_order=0)
Vehicle(id=8, time=30, into_city=0, intercity=1, pas